# Laboratorio 04 --> Clasificación de Malware

Universidad del Valle de Guatemala

Luis Pedro Cuéllar - 18220


### Parte I --> Análisis de un Malware


In [1]:
import os
import glob
import pefile
import hashlib
import numpy as np
import pandas as pd

In [6]:
PATH = "MALWR2"

all_files = glob.glob("{}/*".format(PATH))
all_files

['MALWR2/sample_qwrty_dk2', 'MALWR2/sample_vg655_25th.exe']


#### **Análisis Estático**


In [19]:
df = []

for i in all_files:
    dll_calls = []
    api_calls = []
    
    pe = pefile.PE(i)
    for entry in pe.DIRECTORY_ENTRY_IMPORT:
        dll_calls.append(entry.dll.decode("utf-8"))
        
        for call in entry.imports:
            api_calls.append(call.name.decode("utf-8"))
    
    df.append(dll_calls)
    df.append(api_calls)
    
    print("\n")
    print("FILE NAME :\n\t {}".format(i))
    print("DLL CALLS :\n\t {}".format(dll_calls))
    print("API_CALLS :\n\t {}".format(api_calls))



FILE NAME :
	 MALWR2/sample_qwrty_dk2
DLL CALLS :
	 ['KERNEL32.DLL', 'MSVCRT.dll', 'SHELL32.dll', 'USER32.dll', 'WS2_32.dll']
API_CALLS :
	 ['LoadLibraryA', 'ExitProcess', 'GetProcAddress', 'VirtualProtect', 'atol', 'SHChangeNotify', 'LoadStringA', 'closesocket']


FILE NAME :
	 MALWR2/sample_vg655_25th.exe
DLL CALLS :
	 ['KERNEL32.dll', 'USER32.dll', 'ADVAPI32.dll', 'MSVCRT.dll']
API_CALLS :
	 ['GetFileAttributesW', 'GetFileSizeEx', 'CreateFileA', 'InitializeCriticalSection', 'DeleteCriticalSection', 'ReadFile', 'GetFileSize', 'WriteFile', 'LeaveCriticalSection', 'EnterCriticalSection', 'SetFileAttributesW', 'SetCurrentDirectoryW', 'CreateDirectoryW', 'GetTempPathW', 'GetWindowsDirectoryW', 'GetFileAttributesA', 'SizeofResource', 'LockResource', 'LoadResource', 'MultiByteToWideChar', 'Sleep', 'OpenMutexA', 'GetFullPathNameA', 'CopyFileA', 'GetModuleFileNameA', 'VirtualAlloc', 'VirtualFree', 'FreeLibrary', 'HeapAlloc', 'GetProcessHeap', 'GetModuleHandleA', 'SetLastError', 'VirtualPro

**1. ¿Qué  diferencias  observa  entre  los  ejemplos?  ¿Existe  algún  indicio 
sospechoso en la cantidad de DLLs y las APIs llamadas?**

Entre los dos archivos que se extrajeron, se puede observar que la cantidad de llamadas en el segudo archivo aumenta significativamente. En el segundo archivo podemos observar llamas a API como la de abrir, escribir y cerrar un archivo. También, se obtiene varios path, crea archivos con el path, con el timestamp y otras llamadas a APIs.


In [29]:
df = []

for i in all_files:
    e_magic = []
    signature = []
    sections = []
    
    pe = pefile.PE(i)
    
    print("\nFILE NAME :\n\t {}".format(i))
    
    e_magic.append(pe.DOS_HEADER.e_magic)
    e_magic.append(hex(pe.DOS_HEADER.e_magic))
    
    print("E_MAGIC :\n\t {}".format(e_magic[0]))
    print("E_MAGIC (HEX):\n\t {}".format(e_magic[1]))
    
    signature.append(pe.NT_HEADERS.Signature)
    signature.append(hex(pe.NT_HEADERS.Signature))
    
    print("SIGNATIURE :\n\t {}".format(signature[0]))
    print("SIGNATIURE (HEX):\n\t {}".format(signature[1]))
    
    for section in pe.sections:
        sections.append(section.Name.decode("utf-8"))
        sections.append(hex(section.VirtualAddress))
        sections.append(hex(section.Misc_VirtualSize))
        sections.append(hex(section.SizeOfRawData))
        
        print(section.Name.decode('utf-8'))
        print("\tVirtual Address: " + hex(section.VirtualAddress))
        print("\tVirtual Size: " + hex(section.Misc_VirtualSize))
        print("\tRaw Size: " + hex(section.SizeOfRawData))
    
    df.append(e_magic)
    df.append(signature)
    df.append(sections)


FILE NAME :
	 MALWR2/sample_qwrty_dk2
E_MAGIC :
	 23117
E_MAGIC (HEX):
	 0x5a4d
SIGNATIURE :
	 17744
SIGNATIURE (HEX):
	 0x4550
UPX0    
	Virtual Address: 0x1000
	Virtual Size: 0x5000
	Raw Size: 0x0
UPX1    
	Virtual Address: 0x6000
	Virtual Size: 0x1000
	Raw Size: 0x1000
.rsrc   
	Virtual Address: 0x7000
	Virtual Size: 0x1000
	Raw Size: 0x200

FILE NAME :
	 MALWR2/sample_vg655_25th.exe
E_MAGIC :
	 23117
E_MAGIC (HEX):
	 0x5a4d
SIGNATIURE :
	 17744
SIGNATIURE (HEX):
	 0x4550
.text   
	Virtual Address: 0x1000
	Virtual Size: 0x69b0
	Raw Size: 0x7000
.rdata  
	Virtual Address: 0x8000
	Virtual Size: 0x5f70
	Raw Size: 0x6000
.data   
	Virtual Address: 0xe000
	Virtual Size: 0x1958
	Raw Size: 0x2000
.rsrc   
	Virtual Address: 0x10000
	Virtual Size: 0x349fa0
	Raw Size: 0x34a000


**2. ¿Qué significa que algunas secciones tengan como parte de su nombre “upx”?**

Significa que este paquete se encuentra enpaquetado.

In [31]:
os.system("upx -d {}".format(all_files[0])) 

                       Ultimate Packer for eXecutables
                          Copyright (C) 1996 - 2020
UPX 3.96        Markus Oberhumer, Laszlo Molnar & John Reiser   Jan 23rd 2020

        File size         Ratio      Format      Name
   --------------------   ------   -----------   -----------
      8192 <-      5632   68.75%    win32/pe     sample_qwrty_dk2

Unpacked 1 file.


0

In [33]:
df = []

for i in all_files:
    dll_calls = []
    api_calls = []
    
    pe = pefile.PE(i)
    for entry in pe.DIRECTORY_ENTRY_IMPORT:
        dll_calls.append(entry.dll.decode("utf-8"))
        
        for call in entry.imports:
            api_calls.append(call.name.decode("utf-8"))
    
    df.append(dll_calls)
    df.append(api_calls)
    
    print("\n")
    print("FILE NAME :\n\t {}".format(i))
    print("DLL CALLS :\n\t {}".format(dll_calls))
    print("API_CALLS :\n\t {}".format(api_calls))



FILE NAME :
	 MALWR2/sample_qwrty_dk2
DLL CALLS :
	 ['KERNEL32.DLL', 'MSVCRT.dll', 'SHELL32.dll', 'USER32.dll', 'WS2_32.dll']
API_CALLS :
	 ['CloseHandle', 'WaitForSingleObject', 'CreateEventA', 'ExitThread', 'Sleep', 'GetComputerNameA', 'CreatePipe', 'DisconnectNamedPipe', 'TerminateProcess', 'WaitForMultipleObjects', 'TerminateThread', 'CreateThread', 'CreateProcessA', 'DuplicateHandle', 'GetCurrentProcess', 'ReadFile', 'PeekNamedPipe', 'SetEvent', 'WriteFile', 'SetProcessPriorityBoost', 'SetThreadPriority', 'GetCurrentThread', 'SetPriorityClass', 'lstrcatA', 'lstrcpyA', 'GetEnvironmentVariableA', 'GetShortPathNameA', 'GetModuleFileNameA', 'GetStartupInfoA', 'GetModuleHandleA', '_controlfp', '_beginthread', '_strnicmp', 'sprintf', 'atol', 'strchr', 'free', 'malloc', '_exit', '_XcptFilter', 'exit', '_acmdln', '__getmainargs', '_initterm', '__setusermatherr', '_adjust_fdiv', '__p__commode', '__p__fmode', '__set_app_type', '_except_handler3', '_itoa', 'ShellExecuteExA', 'SHChangeNotif

In [32]:
df = []

for i in all_files:
    e_magic = []
    signature = []
    sections = []
    
    pe = pefile.PE(i)
    
    print("\nFILE NAME :\n\t {}".format(i))
    
    e_magic.append(pe.DOS_HEADER.e_magic)
    e_magic.append(hex(pe.DOS_HEADER.e_magic))
    
    print("E_MAGIC :\n\t {}".format(e_magic[0]))
    print("E_MAGIC (HEX):\n\t {}".format(e_magic[1]))
    
    signature.append(pe.NT_HEADERS.Signature)
    signature.append(hex(pe.NT_HEADERS.Signature))
    
    print("SIGNATIURE :\n\t {}".format(signature[0]))
    print("SIGNATIURE (HEX):\n\t {}".format(signature[1]))
    
    for section in pe.sections:
        sections.append(section.Name.decode("utf-8"))
        sections.append(hex(section.VirtualAddress))
        sections.append(hex(section.Misc_VirtualSize))
        sections.append(hex(section.SizeOfRawData))
        
        print(section.Name.decode('utf-8'))
        print("\tVirtual Address: " + hex(section.VirtualAddress))
        print("\tVirtual Size: " + hex(section.Misc_VirtualSize))
        print("\tRaw Size: " + hex(section.SizeOfRawData))
    
    df.append(e_magic)
    df.append(signature)
    df.append(sections)


FILE NAME :
	 MALWR2/sample_qwrty_dk2
E_MAGIC :
	 23117
E_MAGIC (HEX):
	 0x5a4d
SIGNATIURE :
	 17744
SIGNATIURE (HEX):
	 0x4550
.text   
	Virtual Address: 0x1000
	Virtual Size: 0xea6
	Raw Size: 0x1000
.rdata  
	Virtual Address: 0x2000
	Virtual Size: 0x67e
	Raw Size: 0x800
.data   
	Virtual Address: 0x3000
	Virtual Size: 0x628
	Raw Size: 0x200
.rsrc   
	Virtual Address: 0x4000
	Virtual Size: 0x80
	Raw Size: 0x200

FILE NAME :
	 MALWR2/sample_vg655_25th.exe
E_MAGIC :
	 23117
E_MAGIC (HEX):
	 0x5a4d
SIGNATIURE :
	 17744
SIGNATIURE (HEX):
	 0x4550
.text   
	Virtual Address: 0x1000
	Virtual Size: 0x69b0
	Raw Size: 0x7000
.rdata  
	Virtual Address: 0x8000
	Virtual Size: 0x5f70
	Raw Size: 0x6000
.data   
	Virtual Address: 0xe000
	Virtual Size: 0x1958
	Raw Size: 0x2000
.rsrc   
	Virtual Address: 0x10000
	Virtual Size: 0x349fa0
	Raw Size: 0x34a000


**3. ¿en que categoría sospechosas pueden clasificarse estos ejemplos en base a algunas de las 
llamadas a las APIs que realizan? Muestre una tabla con las APIs sospechosas y la categoría de malware que el paper propone**

| File Name             | Behaviour   | Malware Category     | API Call               |
|-----------------------|-------------|----------------------|------------------------|
| sample_vg655_25th.exe | Behaviour 2 | Copy/Delete Files    | CloseHandle            |
|                       | Behaviour 3 | Get File Information | GetFileSize            |
|                       | Behaviour 5 | Read/Write Files     | CloseHandle, WriteFile |
| sample_qwrty_dk2      | Behaviour 2 | Copy/Delete Files    | CloseHandle            |
|                       | Behaviour 5 | Read/Write Files     | CloseHandle, WriteFile |

**4. Para el archivo “sample_vg655_25th.exe” obtenga el HASH en base al algoritmo SHA256.**

In [35]:
hs = hashlib.sha256(all_files[1].encode('utf-8')).hexdigest()
hs

'e6d7d2eb6ca74ef22c136c33293d911f0d33edac89396129723f30c02490e13f'

**5. Para el archivo “sample_vg655_25th.exe”, ¿cuál es el propósito de la DLL ADVAPI32.dll?**


Advapi32.dll es un archivo DLL avanzado de API base de Windows 32; es una biblioteca de servicios API que admite llamadas de registro y seguridad.

**6. Para  el  archivo  “sample_vg655_25th.exe”,  ¿cuál  es  el  propósito  de  la  API 
CryptReleaseContext?**

Esta función se utiliza para liberar un identificador a un proveedor de servicios criptográficos (CSP) y un contenedor de claves.
Esto debe realizarse cuando la aplicación termine de usar el CSP. Después de llamar a esta función, el identificador de CSP especificado por el parámetro hProv ya no es válido. Esta función no destruye ni el contenedor de claves ni ningún par de claves.

https://docs.microsoft.com/en-us/previous-versions/ms936171(v=msdn.10)

**7. Con  la  información  recopilada  hasta  el  momento,  indique  para  el  archivo “sample_vg655_25th.exe” si es sospechoso o no, y cual podría ser su propósito.**

Por el momento creo que el archivo es sospechoso. Según las llamadas que investigamos, parecían tener una "mala intención". El propósito del archivo podría ser en obtener información y crear archivos, también hay varias llamadas a APIs que crean procesos o eventos.


#### **Análisis Dinámico**


**8. Utilice la plataforma de análisis dinámico https://www.hybrid-analysis.com y cargue el archivo 
“sample_vg655_25th.exe”. ¿Se corresponde el HASH de la plataforma con el generado? ¿Cuál 
es el nombre del malware encontrado? ¿En que consiste este malware?**

El HASH de la plataforma no coincide con el generado. El nombre de este malware es "owo_im_not_ransomware_xd.exe". Este malware consiste en un Trojano. según la página, puede tener aceso a nuestra máquina de forma remota, puede ser spyware, spawnea muchos procesos, escribe datos a un proceso remoto, consulta el kernel para información del sistema, elimina archivos, entre otros.

**9. Muestre las capturas de pantalla sobre los mensajes que este malware presenta a usuario. 
¿Se corresponden las sospechas con el análisis realizado en el punto 7?**

Los screenshots se encuentran en la carpeta llamada "ad_9". Ciertas cosas sí se lograron identificar en la pregunta 7, como el obtener información de archivos, crear archivos y proceos.


### Parte II --> Clasificación de un malware en base a secuencia de llamadas a las APIs



In [41]:
df = pd.read_csv("VirusSample.csv")

In [42]:
df.head()

,file,api,class
0,7ff49f2f0912352416b05c010f35f402cc79feed,"IntersectRect,GetCurrentProcess,GetVersion",Virus
1,50cc6c99ec285d0db45dde07d8fdc18d9098c5b6,"GetCaretBlinkTime,CountClipboardFormats,GetCon...",Virus
2,f77c6bd4aebacd1a01d02e0cb20642ebf2d32929,"VarR8Pow,GetClipboardViewer,GetInputDesktop,Ge...",Virus
3,349c367c5b88fbb6cafae5d7109588d7250e16b5,"SetTraceCallback,CopyAcceleratorTableW,GetProc...",Virus
4,021f4aa86b520e1d606ab26699c35546bcd00c27,"SHLoadNonloadedIconOverlayIdentifiers,VarUI8Fr...",Virus


In [ ]:
TRAIN_RATIO = 0.7
TEST_RATIO = 0.3

